# 09_04: Estimating sampling errors with Monte Carlo

In [ ]:
import math
import collections
import dataclasses
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as pp

In [ ]:
poll = pd.read_csv('poll.csv')
poll.head()

In [ ]:
poll.value_counts(normalize=True)

In [ ]:
def makepoll(nsample=1000, brown=0.51):
    return pd.DataFrame({'vote': np.random.choice(['Brown', 'Green'], nsample, p=[brown, 1.0 - brown])})

In [ ]:
makepoll().value_counts(normalize=True)

In [ ]:
makepoll().value_counts(normalize=True)

In [ ]:
def makedist(ntries=2000, nsample=1000, brown=0.51):
    return pd.DataFrame([makepoll(nsample, brown).value_counts(normalize=True) for i in range(ntries)])

In [ ]:
counts = makedist()

In [ ]:
counts

In [ ]:
pp.figure(figsize=(4,3))
pp.hist(counts['Brown'], histtype='step', bins=20, density=True);

In [ ]:
counts.Brown.describe()

In [ ]:
pp.figure(figsize=(4,3))
pp.hist(makedist(ntries=2000, nsample=1000, brown=0.55)['Brown'], histtype='step', bins=20, density=True);

In [ ]:
pp.figure(figsize=(4,3))
pp.hist(makedist(ntries=2000, nsample=1000, brown=0.55)['Brown'], histtype='step', bins=20, density=True, label='1,000')
pp.hist(makedist(ntries=2000, nsample=10000, brown=0.55)['Brown'], histtype='step', bins=20, density=True, label='10,000')
pp.legend();

In [ ]:
makedist(ntries=2000, nsample=1000, brown=0.55)['Brown'].quantile(0.025)

In [ ]:
makedist(ntries=2000, nsample=1000, brown=0.54)['Brown'].quantile(0.025)

In [ ]:
makedist(ntries=2000, nsample=1000, brown=0.47)['Brown'].quantile(0.975)

In [ ]:
makedist(ntries=2000, nsample=1000, brown=0.48)['Brown'].quantile(0.975)